# Лабораторная работа №6

## Применение сверточных нейронных сетей (многоклассовая классификация)

Набор данных для распознавания языка жестов, который состоит из изображений размерности 28x28 в оттенках серого (значение пикселя от 0 до 255).

Каждое из изображений обозначает букву латинского алфавита, обозначенную с помощью жеста (изображения в наборе данных в оттенках серого).

Обучающая выборка включает в себя 27,455 изображений, а контрольная выборка содержит 7172 изображения.

Данные в виде _csv_-файлов можно скачать на сайте _Kaggle_: https://www.kaggle.com/datamunge/sign-language-mnist

### Задание 1

Загрузите данные. Разделите исходный набор данных на обучающую и валидационную выборки.

In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [0]:
BASE_DIR = '/content/drive/My Drive/Colab Files/mo-2'

import sys

sys.path.append(BASE_DIR)

import os

In [0]:
DATA_ARCHIVE_NAME = 'sign-language-mnist.zip'

LOCAL_DIR_NAME = 'sign-language'

In [0]:
from zipfile import ZipFile

with ZipFile(os.path.join(BASE_DIR, DATA_ARCHIVE_NAME), 'r') as zip_:
    zip_.extractall(path = os.path.join(LOCAL_DIR_NAME, 'train'))

In [0]:
TRAIN_FILE_PATH = 'sign-language/train/sign_mnist_train.csv'
TEST_FILE_PATH = 'sign-language/train/sign_mnist_test.csv'

In [0]:
import pandas as pd

train_df = pd.read_csv(TRAIN_FILE_PATH)
test_df = pd.read_csv(TEST_FILE_PATH)

In [0]:
all_df = pd.concat([train_df, test_df])

In [8]:
print(all_df.shape)

(34627, 785)


In [0]:
IMAGE_DIM = 28

In [0]:
def row_to_label(_row):
    return _row[0]

def row_to_one_image(_row):
    return _row[1:].values.reshape((IMAGE_DIM, IMAGE_DIM, 1))

In [0]:
def to_images_and_labels(_dataframe):

    llll = _dataframe.apply(lambda row: row_to_label(row), axis = 1)
    mmmm = _dataframe.apply(lambda row: row_to_one_image(row), axis = 1)

    data_dict_ = { 'label': llll, 'image': mmmm }

    reshaped_ = pd.DataFrame(data_dict_, columns = ['label', 'image'])

    return reshaped_

In [0]:
all_df_reshaped = to_images_and_labels(all_df)

### Задание 2

Реализуйте глубокую нейронную сеть со сверточными слоями. Какое качество классификации получено? Какая архитектура сети была использована?

Возьмём _LeNet-5_.

In [13]:
! pip install tensorflow-gpu --pre --quiet

! pip show tensorflow-gpu

Name: tensorflow-gpu
Version: 2.2.0rc3
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: google-pasta, keras-preprocessing, wrapt, numpy, h5py, tensorboard, scipy, opt-einsum, tensorflow-estimator, protobuf, six, wheel, termcolor, grpcio, absl-py, gast, astunparse
Required-by: 


In [0]:
from tensorflow.keras.utils import to_categorical
import numpy as np

X = np.asarray(list(all_df_reshaped['image']))

y = to_categorical(all_df_reshaped['label'].astype('category').cat.codes.astype('int32'))

In [0]:
CLASSES_N = y.shape[1]

In [0]:
import tensorflow as tf

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Flatten

model = tf.keras.Sequential()

model.add(Conv2D(6, kernel_size = (5, 5), strides = (1, 1), activation = 'tanh', padding = 'same',
                   input_shape = (IMAGE_DIM, IMAGE_DIM, 1)))
model.add(AveragePooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
model.add(Conv2D(16, kernel_size = (5, 5), strides = (1, 1), activation = 'tanh', padding = 'valid'))
model.add(AveragePooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
model.add(Flatten())
model.add(Dense(120, activation = 'tanh'))
model.add(Dense(84, activation = 'tanh'))
model.add(Dense(CLASSES_N, activation = 'softmax'))

In [0]:
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['categorical_accuracy'])

In [19]:
model.fit(x = X, y = y, epochs = 10, validation_split = 0.15)

Epoch 1/10
920/920 [==============================] - 4s 4ms/step - loss: 0.8570 - categorical_accuracy: 0.7699 - val_loss: 0.2257 - val_categorical_accuracy: 0.9575
Epoch 2/10
920/920 [==============================] - 3s 4ms/step - loss: 0.0546 - categorical_accuracy: 0.9943 - val_loss: 0.0513 - val_categorical_accuracy: 0.9919
Epoch 3/10
920/920 [==============================] - 3s 4ms/step - loss: 0.0288 - categorical_accuracy: 0.9946 - val_loss: 0.0454 - val_categorical_accuracy: 0.9896
Epoch 4/10
920/920 [==============================] - 3s 4ms/step - loss: 0.0082 - categorical_accuracy: 0.9989 - val_loss: 0.0063 - val_categorical_accuracy: 0.9996
Epoch 5/10
920/920 [==============================] - 3s 4ms/step - loss: 0.0015 - categorical_accuracy: 1.0000 - val_loss: 0.0036 - val_categorical_accuracy: 0.9998
Epoch 6/10
920/920 [==============================] - 3s 4ms/step - loss: 9.3112e-04 - categorical_accuracy: 1.0000 - val_loss: 0.0029 - val_categorical_accuracy: 0.9994


За 10 эпох удалось достичь 100%-й точности на валидационной выборке!

### Задание 3

Примените дополнение данных (_data augmentation_). Как это повлияло на качество классификатора?

### Задание 4

Поэкспериментируйте с готовыми нейронными сетями (например, _AlexNet_, _VGG16_, _Inception_ и т.п.), применив передаточное обучение. Как это повлияло на качество классификатора? Можно ли было обойтись без него?

Какой максимальный результат удалось получить на контрольной выборке?